## Machine Learning Problem

The dataset is weather data from the edmonton city-center from 1961 to 1994, with the goal to predict weather "descriptions" (i.e. cloudy, snow, rain etc) from features (temperature, relhumidity, pressure)


### Classifiers
I choose 3 classifiers + the baseline to explore the performance of different algorithms, all using the
scikit-learn implementations
\begin{itemize}
    \item There's the obvious baseline of a $\frac{1}{k}$ zero-classifier, which we will hopefully outperform
    \item Linear regression as a basic model
    \item SVM
    \item and a relatively simple multi-layer perceptron neural net from SKLearn
\end{itemize}

In [19]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import svm
from sklearn.neural_network import MLPClassifier

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# tqdm loading bars
from tqdm.notebook import tqdm

In [24]:
data = pd.read_csv("weather_data.csv")
data[['TempC','R']]

,\tSec,DateTime,TempC,DewPtC,RelHumPct,WDir10Deg,WSpdKph,PkPa,Weather
0,94694400,1953-01-01 00:00,-8.3,-8.9,96.0,14.0,10.0,92.59,Clear
1,94698000,1953-01-01 01:00,-8.9,-10.0,90.0,23.0,10.0,92.64,Clear
2,94701600,1953-01-01 02:00,-8.9,-10.0,90.0,20.0,6.0,92.71,Clear
3,94705200,1953-01-01 03:00,-10.0,-10.6,93.0,23.0,6.0,92.79,Clear
4,94708800,1953-01-01 04:00,-11.1,-12.2,92.0,23.0,10.0,92.84,Clear
...,...,...,...,...,...,...,...,...,...
720086,2214327600,2020-03-02 19:00,1.7,-6.4,55.0,25.0,4.0,92.19,NaN
720087,2214331200,2020-03-02 20:00,-0.1,-5.8,65.0,22.0,3.0,92.16,NaN
720088,2214334800,2020-03-02 21:00,0.9,-5.7,61.0,20.0,9.0,92.14,NaN
720089,2214338400,2020-03-02 22:00,0.7,-6.4,59.0,20.0,10.0,92.05,NaN


In [21]:
X = data[['TempC', 'RelHumPct', "PkPa"]]
y = data[['Weather']]

### Train, validation, test
I implement a simple training, validation and test data splitting function here, with a
default value of 60%, 20%, 20% for the three respectively. Different values can be
passed in as needed.

In [22]:
def train_validation_test_splits(X, y, sizes = [.6, .2, .2], random_seed = 314):
    """
    :X: the usual X as above
    :y: the usual y as above
    :sizes: the split sizes [train, validation, test]
    :random_seed: the random seed for shuffle operations done in this function to improve
    repeatability
    :return: X_train, y_train, X_val, y_val, X_test, y_test
    """
    